In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 

In [ ]:
pip install datasets

In [ ]:
pip install accelerate

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.9 MB/s eta 0:00:00


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [ ]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "test_detection")

print(data.keys())

data_de_en = data['test_detection_de_en'].to_pandas()
data_en_de = data['test_detection_en_de'].to_pandas()
data_fr_en = data['test_detection_fr_en'].to_pandas()
data_en_fr = data['test_detection_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating test_detection_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['test_detection_de_en', 'test_detection_en_de', 'test_detection_fr_en', 'test_detection_en_fr'])


,id,langpair,source,hyp1,hyp2
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In developed countries, offering luxury guesth..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice elsewhere in the UK, b..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"“Based on this fossil, we can say that the sep...","“Based on this fossil, we can say that the sep..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","An Avenger class anti-mining ship, the ship wa...","An Avenger class anti-mining ship, the ship wa..."


,id,langpair,source,hyp1,hyp2
0,0,en-de,He graduated from the College of Arts & Scienc...,Er absolvierte 1950 das College of Arts & Scie...,Er absolvierte 1950 das College of Arts & Scie...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten der kleineren Inseln sind unabhäng...,"Die meisten sind unabhängige Nationen, oder mi..."
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die immer...","Aber es gibt viele Dinge über Vögel, die wie e..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...",Die neuseeländische Polizei hatte Schwierigkei...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","""Das Plateau von Giza oder """"Nekropolis von Gi...","""Das Plateau von Giza oder """"Nekropolis von Gi..."


In [ ]:
#load the trial data for all language pairs
trial_ds = load_dataset("Eloquent/HalluciGen-Translation", name="trial")
print(trial_ds.keys())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

dict_keys(['trial_de_en', 'trial_en_de', 'trial_fr_en', 'trial_en_fr'])


In [ ]:
trial_ds_de_en = trial_ds['trial_de_en'].to_pandas()
trial_ds_en_de = trial_ds['trial_en_de'].to_pandas()
trial_ds_fr_en = trial_ds['trial_fr_en'].to_pandas()
trial_ds_en_fr = trial_ds['trial_en_fr'].to_pandas()

display(trial_ds_de_en.head())
display(trial_ds_en_de.head())

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems ge...,"Die Tage im Sommer können zu Problemen führen,...",Die sehr langen Tage im Sommer können zu Probl...,addition,hyp2
1,1,en-de,The vertical clearance under the bridge is 15 ...,Der vertikale Abstand unter der Brücke beträgt...,Der vertikale Abstand unter der Brücke beträgt...,date,hyp1
2,2,en-de,William Llewelyn Williams known as Llewelyn Wi...,"William Williams, bekannt als Llewelyn William...","Williajm Llewelyn Williams, bekannt als Llewel...",named-entity,hyp2
3,3,en-de,The single was released on Radio Airplay in It...,Die Single wurde am 12. Oktober 212 in Italien...,Die Single wurde am 12. Oktober 2012 in Italie...,number,hyp1
4,4,en-de,Water is spilling over the levee in a section ...,Wasser fließt über den Deich in einem Abschnit...,Wasser fließt über die Leibe in einem Abschnit...,natural,hyp2


## Load Model

In [ ]:
#define the Model
model = "google/gemma-7b-it"
# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt 1

In [ ]:
def generate_label(source, hyp1, hyp2, translation_language):

  answer_format = {"label": " "}

  # prompt for translation generation task
  user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.
      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      label: 'hyp2'
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  label = outputs[0]["generated_text"][len(prompt):]
  print(label)

  return label

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

### Test

In [ ]:
source = trial_ds_de_en.iloc[0]["source"]
hyp1 = trial_ds_de_en.iloc[0]["hyp1"]
hyp2 = trial_ds_de_en.iloc[0]["hyp2"]
hallucination = generate_label(source, hyp1, hyp2, "English")
json_data = extract_json_data(hallucination)
print(json_data)


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


```python
{'label': 'hyp2'}

Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
hyp1 : In the phase, wandering ants march at night and rest during the day.
hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.

Result: {'label': 'hyp2'}
```

The text "Marschieren Wanderameisen bei Nacht und machen tagsüber Rast" is more similar to "hyp2" than "hyp1". Therefore, the result is {'label': 'hyp2'}.
{'label': 'hyp2'}


In [ ]:
trial_ds_de_en.iloc[0]["label"]


'hyp2'

# Test with Trial dataset

### Prompt 1

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label").
Provide the result in the following format: {"label": ""}.

Src: {source}


hyp1 : {hyp1}


hyp2 : {hyp2}


Result:


In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = generate_label(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/trial_results_gemma_de_en_test2.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Detection

**Source:** In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.

**Hyp1:** In the phase, wandering ants march at night and rest during the day.

**Hyp2:** In the nomadic phase, wandering ants march at night and rest during the day.

**Label:** 'hyp2'

**Type:** Addition

**Explanation:**

The source text describes the behavior of wandering ants. It states that they march at night and rest during the day. The text does not mention the "nomadic phase". Therefore, the addition of the phrase "nomadic phase" in hyp2 is hallucinated.

**Therefore, the result is:**

```
{'label': 'hyp2', 'type': 'addition'}
```


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Result:**

{'label': 'hyp2', 'type': 'addition'}

**Explanation:**

The source text describes an incident where front singer Steven Tyler injured himself during a performance on August 5th and consequently canceled the tour. The text clearly states the date of the incident as August 5th. Therefore, the hallucination is 'hyp2', which incorrectly states the date of the incident as December 5th. The type of hallucination is 'addition', as the hallucination involves adding the incorrect date to the text.
**Result:**

{'label': 'hyp2', 'type': 'addition'}

**Explanation:**

The source text describes a situation where the auditor caught someone cheating. The text does not provide any information about the gender of the developer, therefore, the hallucination is hyp2. The hallucination type is addition, as the text adds the phrase "cheating" to the end of the sentence.
## Detect Hallucination

**Source:**

Er wurde in aufeinanderfolgenden Coups durch Olusegun Obasanjo (1975) und Murtala Moh

In [ ]:
df = pd.read_csv(csv_filename)

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/trial_report_gemma_de_en_test2.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,NaN
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp2
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,NaN
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,NaN
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,NaN
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,NaN
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,NaN
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,NaN
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2


ValueError: Classification metrics can't handle a mix of binary and unknown targets

#prompt 2

In [ ]:
def generate_label(source, hyp1, hyp2, translation_language):

  answer_format = {"label": "",
                   "type": ""}

  # prompt for translation generation task
  user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to the language of "src".
    Then, compare these translations with "src" to detect which of the two hypotheses is hallucinated ("label") and what type of hallucination ("type") it is.
    Provide the result in the following format: {answer_format}.

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  label = outputs[0]["generated_text"][len(prompt):]
  print(label)

  return label

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = generate_label(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/trial_results_gemma_de_en_promt2.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Translation:**

**src:** In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.

**hyp1:** In the phase, wandering ants march at night and rest during the day.

**hyp2:** In the nomadic phase, wandering ants march at night and rest during the day.

**Result:**

{'label': 'hyp2', 'type': 'Nomadismus'}

**Explanation:**

* The translation of "hyp1" is very similar to the original text "src", except for the use of the word "nomad" instead of "Marschieren".
* The translation of "hyp2" is more accurate, as it uses the term "nomad" to describe the phase in which the ants are wandering.
* Therefore, the hypothesis that is hallucinated is "hyp2", and the type of hallucination is "Nomadismus".


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Result:**
{'label': 'hyp1', 'type': 'Date mismatch'}

**Explanation:**

* **Translation of hyp1:** "After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour."

* **Translation of hyp2:** "After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 5, they canceled the tour."

* **Comparison with src:** The translation of hyp1 matches the source text more closely in terms of the date ("December 5") and the use of the word "during". The translation of hyp2 does not match the source text as closely, as it uses the date "Aug. 5" instead of "December 5".

* **Hallucination:** Hyp1 is hallucinated because it incorrectly states the date of the injury as "December 5", which is not supported by the source text.

*
**Translation:**

**src:** Der Wirtschaftsprüfer hat die Entwicklerin beim Schummeln erwischt.

**hyp1:** The auditor caught the male developer cheating.

**hyp2:** T

In [ ]:
df = pd.read_csv(csv_filename)

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/£.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp1
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.60      0.75         5
        hyp2       0.71      1.00      0.83         5

    accuracy                           0.80        10
   macro avg       0.86      0.80      0.79        10
weighted avg       0.86      0.80      0.79        10



# get label for dataset

In [ ]:
def generate_label(source, hyp1, hyp2, translation_language):

  answer_format = {"label": ""}

  # prompt for translation generation task
  user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to the language of "src".
    Then, compare these translations with "src" to detect which of the two hypotheses is hallucinated ("label") and what type of hallucination ("type") it is.
    Provide the result in the following format: {answer_format}.

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    '''


  messages = [
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
  )

  #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  label = outputs[0]["generated_text"][len(prompt):]
  print(label)

  return label

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = generate_label(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

# DE - EN

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/results_gemma_de_en.csv"
skipped_rows = save_trial_results(data_de_en, csv_filename, "English")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to

Skipping row 64 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 74 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


# EN - DE

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/results_gemma_en_de_fix.csv"
skipped_rows = save_trial_results(data_en_de, csv_filename, "German")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


**Result:**

```
{'label': 'hyp2', 'type': 'grammatical'}
```

**Explanation:**

* **Translation of hyp1:** "Er absolvierte 1950 das College of Arts & Sciences der University of Virginia und war ein bedeutender Spender dieser Institution."

This translation is grammatically correct, but it does not match the exact wording of "src".

* **Translation of hyp2:** "Er absolvierte 1950 das College of Arts & Sciences und war ein bedeutender Spender dieser Institution."

This translation is an exact match of "src".

* **Comparison with src:**

Hyp1 differs from "src" in the use of the phrase "der University of Virginia". Hyp2 is an exact match of "src".

* **Hallucination:**

Hyp2 is hallucinated because it is an exact match of "src", which is not possible given the context of the text. The halluc
**Result:**

```
{'label': 'hyp1', 'type': 'Lexical Hallucination'}
```

**Explanation:**

* **Translation of hyp1:** "Die meisten der kleineren Inseln sind unabhängige Nationen, oder mit Frankreich 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Result:

{'label': 'hyp1', 'type': 'grammatical'}

**Explanation:**

* **Hyp1:** Am 24. Oktober 1759 unterzeichnete Arthur Guinness eine 9.000-jährige Miete für die St. James' Gate Brewery in Dublin, Irland.
This translation is grammatically incorrect. The correct translation is: Am 24. Oktober 1759 unterzeichnete Arthur Guinness eine 9.000-jährige Miete für die St. James' Gate Brewery in Dublin, Irland.

* **Hyp2:** Am 24. Sept. 1759 unterzeichnete Arthur Guinness eine 9.000-jährige Miete für die St. James' Gate Brewery in Dublin, Irland.
This translation is grammatically correct, but the date is incorrect. The correct date is 24. Oktober 1759, not 24. Sept


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

**Result:**

```
{'label': 'hyp1', 'type': 'grammatical'}
```

**Explanation:**

* **Translation of hyp1:** Sie hat ihre Schwester immer entmutigt.
This translation is grammatically correct, but it does not match the meaning of the source text.
* **Translation of hyp2:** Sie hat ihrer Schwester immer Mut gemacht.
This translation is also grammatically correct, but it does not match the meaning of the source text either.

**Comparison with src:**

The source text is "She always encouraged her sister".

Hyp1 is more similar to the source text in terms of grammar and meaning. However, the use of the word "entmutigt" is not appropriate in this context.

Hyp2 is not similar to the source text in terms of grammar or meaning. The use of the word "Mut" is not appropriate in this context.

Therefore, the hallucination is hyp1, and the type
**Result:**

```
{'label': 'hyp1', 'type': 'overgeneralization'}
```

**Explanation:**

* **Translation of hyp1:** "I wurde klar, dass ich vielleicht schnell

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Result:**

{'label': 'hyp1', 'type': 'overgeneralization'}

**Explanation:**

* **Translation of hyp1:** The translation of hyp1 back to the language of src is similar to the original text, but with some minor differences in wording and sentence structure.
* **Comparison with src:** When comparing the translation of hyp1 with src, it is found that hyp1 captures the main ideas of src, but it does not include the phrase "particular selected individuals".
* **Hallucination:** The hallucination in hyp1 is the overgeneralization of patterns from a few individuals to make conclusions about human nature and disease.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


**Result:**

```
{'label': 'hyp1', 'type': 'grammatical'}
```

**Explanation:**

* **Translation of hyp1:** "Also meine erste Frage: wo ist Google und wohin geht sie?"
* **Translation of hyp2:** "Also meine erste Frage: wo ist Google und wohin geht es?"

Both translations are grammatically correct and capture the meaning of the source text. However, the translation of hyp1 is more faithful to the original text, while the translation of hyp2 deviates slightly in terms of word choice and sentence structure.

Therefore, the label for hyp1 is 'hyp1', and the type of hallucination is 'grammatical'.
**Result:**

```
{'label': 'hyp1', 'type': 'grammatical'}
```

**Explanation:**

* **Translation of hyp1:** Ich musste in den Bus einsteigen und ihn nehmen und auf sie warten. This translation is grammatically correct, but it does not match the original sentence exactly.
* **Translation of hyp2:** Ich musste in den Bus einsteigen und ihn nehmen und auf ihn warten. This translation is also grammat

# FR - EN

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/results_gemma_fr_en.csv"
skipped_rows = save_trial_results(data_fr_en, csv_filename, "English")
data_fr_en

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentiall

,id,langpair,source,hyp1,hyp2
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly has appointed a new Electoral Council...,Martelly has appointed a new Provisional Elect...
1,1,fr-en,"est relativement facile en italien, car la pl...","is relatively easy in Italian, as most words a...","Pronunciation is relatively easy in Italian, a..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism can be distinguished from other form...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern Rock had applied for aid due to its e...,Northern had applied for aid due to its exposu...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","Over time, as the population adapts to its new...","Over time, as the new population adapts to its..."
...,...,...,...,...,...
95,95,fr-en,"La couleur de capuchon varie, de marron à jaun...",The cap colorlessness varies from brown to yel...,"The cap color varies from brown to yellow, oft..."
96,96,fr-en,La rivière Arieşul Mare est un affluent de la ...,The Arieşul Mare River is a distributary of th...,The Arieşul Mare River is a tributary of the r...
97,97,fr-en,"Born est né en Allemagne de parents juifs, le ...","Born was born in Germany to Jewish parents, th...","Born in Germany was born to Jewish parent, the..."
98,98,fr-en,"À proximité se trouve la rivière Wallowa, un a...","Nearby is the Wallowa River, a distributary of...","Nearby is the Wallowa River, a tributary of th..."


# EN - FR

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/gemma/results_gemma_en_fr.csv"
skipped_rows = save_trial_results(data_en_fr, csv_filename, "French")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to

Skipping row 79 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Skipping row 88 due to error in data extraction.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
